## STACKS pipeline


This notebook goes through the STACKS pipeline *with* the additional filtering steps (after both `cstacks` and `populations`) that are used by our lab. Credit for the `python` and `bash` scripts run for the additional filtering steps to Marine Brieuc, Dan Drinan, Charlie Waters, and Eleni Petrou. 

Their scripts were modified here for my data; for copies of their original scripts, refer to my evernote notebook [here](http://www.evernote.com/l/AorJPQvCni5C5peVjdcf6V_sYE87dAP_h8c/), or ask for their evernote notebooks. 

The command line code in this notebook was not run on my data; it is here for reference. The other Jupyter notebooks in this repo contain command line code that was actually used for data analysis. 

The scripts in the command line code can be found in the [reference scripts]() folder on github. 

<br>
<br>
**PROGRAMS: **
1. [stacks v1.44](http://catchenlab.life.illinois.edu/stacks/) -- see [here](http://www.evernote.com/l/Aoryz9urcLxDMKAbS8TjTle88gzwAKM56og/) for installation description. 
2. FastQC
3. python 2.7.0
4. VMWare Workstation 12 Player, Ubuntu 64-bit -- see [here](http://www.evernote.com/l/Aoryz9urcLxDMKAbS8TjTle88gzwAKM56og/) for installation description. 


### Overview: 

![img](https://github.com/mfisher5/mf-fish546-2016/raw/master/Diagrams/flowchart1.png)
![img](https://github.com/mfisher5/mf-fish546-2016/raw/master/Diagrams/flowchart2.png)


<br>
<br>

### Sequencing data
<br>
Your sequencing data should come back as one (single read) or two (paired end) large gzipped fastq files. You'll want to take a look at them in FastQC...

1. Download and install FastQC as a folder in your root directory. This requires java (see command line below if you do not have java already installed). To install FastQC, go [here](http://www.bioinformatics.babraham.ac.uk/projects/fastqc/). The instructions for installation are in the `INSTALL.txt` file within the fastqc folder. 

In [ ]:
#install latest version of java
sudo apt-get update #updates package index
java -version #checks to make sure java isn't already installed. 
sudo apt-get install default-jre #installs java runtim
sudo apt-get install default-jdk #installs java development kit

(2) Navigate to FastQC folder and open the program

In [ ]:
cd DataAnalysis/fastqc_v0.11.5/FastQC
chmod 755 fastq
./fastqc

(3)   In the pop-up window, select File > Open. From the dropdown menu, select "/" and navigate to the `raw_data` subdirectory. From the "Files" box, select the `fastq.gz` file. 

You can explore from here; one of the most important things to check is per-base sequence quality. You want to see how the sequence quality changes across the length of each read; the quality should decrease towards the end of the read. 

This will give you an idea of how to trim your data in `process_radtags`; see below. 

I like to save my FastQC output so that I can refer back to it. Examples of FastQC output can be found [here](http://www.evernote.com/l/Aop4Hv2efMJKarQTFQ6NtlVES2W7fhalyvQ/) as html files, under "10/7/2016"

### Stacks: `process_radtags` to `ustacks`
<br>
`process_radtags`, `ustacks`, and counting the number of reads per individual to pick samples for `cstacks` can be run straight through using a shell script. Below I'll go through each step separately, and then at the end I'll put in how to generate and run the shell script. 

<br>

### `process_radtags`
<br>

**what does process_radtags do?**

From the manual:  "This program examines raw reads from an Illumina sequencing run and first, checks that the barcode and the RAD cutsite are intact, and demultiplexes the data. If there are errors in the barcode or the RAD site within a certain allowance process_radtags can correct them. Second, it slides a window down the length of the read and checks the average quality score within the window. If the score drops below 90% probability of being correct (a raw phred score of 10), the read is discarded. This allows for some seqeuncing errors while elimating reads where the sequence is degrading as it is being sequenced. By default the sliding window is 15% of the length of the read, but can be specified on the command line (the threshold and window size can be adjusted)."

aka demultiplexes full lane into individuals (*can rename the samples here, from barcode to individual name*), trims the barcodes / sequence length, and does sequence quality control. 
<br>
<br>
**what are the inputs?**

1. raw data: gzipped FASTQ files (`fastq.gz`). 
2. barcodes: text file with a list of barcodes and their corresponding sample names (tab delimited). they don't have to be in order, because process_radtags will search for each barcode at the beginning of every sequence in the raw data. 
<br>
<br>
**what are the outputs?**

*with single read data:* one file. A compilation of all of the retained sequences with a specific barcode. Each file will be named according to the given sample name associated with the barcode. *Note - you can also specify an argument that will put the discarded reads into a file. 
<br>

*with paired end data:* four files. (1) All of the retained forward reads, file extension "`.1.fq.gz`. (2) All of the retained reverse reads, file extension "`.2.fq.gz`. (3) All of the discarded forward reads, file extension "`.rem.1.fq.gz`. (4) All of the discarded reverse reads, file extension "`.rem.2.fq.gz`.
<br>

We use either fasta or gzfasta files as the output. `fastq` files will include the phred score, which we don't need further down the pipeline BUT allow you to look at per base sequence quality in FastQC program. If you are not worried about sequence quality, then you can use `fasta` files instead, which take up less memory. gzipped files will be compressed, so they'll take up less memory but be less accessible. 
<br>
<br>

**running process_radtags**
1. Create subdirectories for the raw data, for the process_radtags output, and for future runs of stacks

In [ ]:
# navigate to root directory
mkdir raw_data
mkdir samplesT142
mkdir stacks

2. Download raw sequencing data into the `raw_data/` folder

3. Create a barcodes .txt file structured: "barcode id" \t "sample name". save into `raw_data/` folder. 

4. return to the root directory.

5. run the process_radtags code. Code inputs: 
        -p (directory to raw data files)
        -P (for paired end data)
        -o (directory for output from the program)
        -b (directory + name of file for barcodes)
        -i (input file type)
        -y (output file type)
        -e (digestion enzyme)
        -E (phred system)
        -r (rescue barcodes and radtags)
        -c (clean the data, remove reads with uncalled bases)
        -q (discard reads with low quality scores)
        -t (trim)
        --filter_illumina (discard reads markedby Illumina's chastity / purity filter.
          
!!! Using `trim`: according to Dan, last base is usually removed b/c it is poor quality; on the above help site response by Julian Catchen, he says most people remove the last 3-5 bases. You should use FastQC to check the per sequence base quality of your raw data to ensure that 3-5 bases is enough (or, if it is too few). You want to trim so that the last few bases are either in the green or beginning yellow zone. **Note that `process_radtags` removes the barcode sequence BEFORE trimming each read, so when the program gets to the "trim" command, the sequence length is actually = (total sequence length in FastQC - 6 bases).**

Example = my sequences are 151bp, and I want to trim the last 3 bases, so I use `-t 142`. 

<br>
!!! Using -`-filter_illumina`: Kristen used this setting to process her data sets, but not many people in this lab do. From           [Julian Catchen](https://groups.google.com/forum/#!topic/stacks-users/huHk_voJ9nE): "Few people use the chastity filter, I personally think it is too strict"


In [ ]:
# for single read data
process_radtags -p raw_data/ \
-i gzfastq -y gzfastq \
-o samplesT142 \
-b raw_data/barcodesL1.txt \
-e sbfI -E phred33 -r -c -q -t 142

In [ ]:
# for paired end data: use the "-P"
process_radtags -p raw_data/ \
-P -i gzfastq -y gzfastq \
-o samplesT142 \
-b raw_data/barcodesL1.txt \
-e sbfI -E phred33 -r -c -q -t 142

(6)  At this point, you should check several of your individuals (at least two per population, or per protocol) to make sure that you trimmed to the correct length. Do this using FastQC, 

### `ustacks`
<br>

**What does ustacks do?**

From the manual: "...will take as input a set of short-read sequences and align them into exactly-matching stacks. Comparing the stacks, it will form a set of loci and detect SNPs at each locus using a maximum likelihood framework."

`ustacks` is used when you don't have a reference genome to align your sequences to. The stacks are made separately for each individual from short-read sequences. 


**what are the inputs?**
1. The gzipped fasta or fastq files that were output from `process_radtags`, now in the "samples" folder. Don't worry if you have both the retained and discarded reads in the "samples" folder; stacks will know which files to use. HOWEVER, if you have paired end data and only want to use the forward reads, you will have to specify `.1` in the sample name. 


**what are the outputs?**
three tsv files per individual: 
1. `.tags.tsv.gz`
2. `.snps.tsv.gz`
3. `.alleles.tsv.gz`

<br>
**running `ustacks`**

Options to run `ustacks`: 

          -t (input file type)
          -f (input file name)
          -r (drop highly repetitive stacks)
          -d (resolve over-merged tags)
          -o (output file path)
          -i (individual ID name)
          -m (minimum depth of coverage - # of sequences at a single locus - to create a stack. usually use 10)
          -M (max. distance between stacks - maximum # of SNPs in a single sequence. usually use 3)
          -p (enable parallel execution)
          -H (disable calling haplotypes from secondary reads - reads that were discarded b/c of minimum depth of coverage can be used by searching for a match across all sequences). Dan uses this, but you don't have to 


You have to run an individual line of `ustacks` code for each sample. An example of one line of code from the root directory: 

In [ ]:
ustacks -t gzfastq \
-f samplesT142/PO010715_06.fq.gz \
-r -d -o stacks \
-i 001 -m 10 -M 3 -p 6

The easiest way to do this is to use a python script that will generate a shell script to run all of this code. If you want to run `ustacks` separately from `process_radtags` and sequence counting, you can extract that chunk of code from the larger shell script below. 

### prepping for `cstacks`

If you DO NOT HAVE A REFERENCE GENOME, you need to choose the individuals that will be used to generate a catalog in `cstacks`. You want the individuals with the greatest number of reads, which we can assume translates to the greatest read depth. 

To find these individuals, you want to count the number of sequences in each individual's `process_radtags` gzipped fasta or fastq file. I do this using a bash pipeline on the command line: 

In [ ]:
#fasta files: count the # of lines in the file and divide by two
cd samplesT142
zcat PO010715_06.fa.gz | wc -l >> FastqReadCounts.txt
#divide output by 2

In [ ]:
#fastq files: you'll need special code for this
cd samplesT142
zcat PO010715_06.fa.gz | awk '((NR-2)%4==0){read=$1;total++;count[read]++}END{for(read in count){if(!max||count[read]>max) {max=count[read];maxRead=read};if(count[read]==1){unique++}};print total}' \
>> FastqReadCounts.txt

The easiest way to do this is to use a python script that will generate a shell script to run all of this code. If you want to run this part separately, you can extract that chunk of code from the larger shell script below. 


After finding the total number of sequences per individual, I copy and paste the sample + total sequence count into an excel spreadsheet, move populations into different columns, and then sort the total number of sequences from highest to lowest. 

I use the 10 individuals from each population with the greatest read depth; if there are fewer than 10 samples in the population, I use all of the individuals. You can use more or less than 10 individuals per population in the catalog; this was determined to be the best number by Charlie (ask to see his barplot). 
<br>
See below for the next step. 
<Br>

### SHELL SCRIPT: `process_radtags` to sequence counts

The following python script will generate a shell that runs all of the steps explained above. For this python script, you will need...

**To Change in the Script: **

(1) Process_radtags

    a. absolute path to folder containing raw data
    
    b. absolute path to folder to place samples in
    
    c. absolute path to folder containing barcode text file
    
    d. arguments for process_radtags

<br>
(2) ustacks

    a. absolute path you want to run ustacks from (should be one directory ABOVE the folder containing your sample fastq files output from process_radtags)
    
    b. relative path to sample fastq files
    
    c. relative path for stacks to put file output 
    
    d. ustacks arguments
    
    *Note: make sure to replace b - d for all iterations of the if-else statement*

<br>
(3) Read counts

    a. relative path from current directory (where you are running ustacks to folder containing fastq sample files. 

<br>

**Arguments for Command Line: **

(1) text file with barcodes and corresponding sample IDs

(2) file name for the new shell script (if including file extension, should be .sh)

In [ ]:
#single read
python radtags_ustacks_genShellSR.py \
barcodesL1.txt \
radtags_ustacks_shell_L1.sh

#paired end: forward reads only
python radtags_ustacks_genShellPE.py \
barcodesL1.txt \
radtags_ustacks_shell_L1.sh

The shell script above will give you **(1)** a folder containing all of the `process_radtags` demultiplexed fastq sample files, **(2)** a folder containing all of the `ustacks` file output, and **(3)** a list of read counts for each of your samples in the `process_radtags` sample folder. 

<br>
Now you'll have to pick the 10 samples from each population with the greatest read depth. You can copy the sample IDs from your barcodes text file into a single column in excel. Then copy all of the read counts from the above generated text file into a single column in excel. Since the code to generate the read count text file is created using the barcodes text file, the order of the sample IDs and the read counts will match. 

<br> 
The next step is to create TWO NEW TEXT FILES. 

1. A file containing the names of your samples to build the `cstacks` catalog, each sample on a new line. 
2. A "population map" file. This contains a list of your samples with their respective populations. format: sampleID "\t" populationID. 

You can find examples of these two files [here]()